In [3]:
import os
import json
import re

#from tqdm import tqdm

#from keybert import KeyBERT
#kw_model = KeyBERT()

superseg, tiage

In [38]:
ngram_range = (1, 1)
top_n = 5
superseg_path = './datasets/superseg'
splits = ['train', 'validation', 'test']


In [39]:
for split in tqdm(splits):
    with open(os.path.join(superseg_path, f'segmentation_file_{split}.json')) as f:
        data = json.load(f)
    dataset = {'data' : []}
    for idx, current_dialog in enumerate(tqdm(data['dial_data']['superseg-v2'])):
        current_topic_utterances = [] 
        current_topic_id = 0
        for item in current_dialog['turns']:
            if item['topic_id'] == current_topic_id:
                current_topic_utterances.append(item['utterance'].strip())
            else:
                keywords = kw_model.extract_keywords(' '.join(current_topic_utterances), top_n=top_n,  keyphrase_ngram_range=ngram_range, stop_words='english') 
                keywords = [word[0] for word in keywords]
                dataset['data'].append({"text": "\n".join(current_topic_utterances),
                                        "topic_id": current_topic_id,
                                        "keywords": ', '.join(keywords),
                                        "dialogue_id": idx})
                current_topic_id += 1
                current_topic_utterances = [item['utterance'].strip()]

    with open(f'./data/superseg_{split}.json', 'w') as f:
        json.dump(dataset, f)   

100%|██████████| 3/3 [07:06<00:00, 142.16s/it]


In [36]:
ngram_range = (1, 1)
top_n = 5
superseg_path = './datasets/tiage'
splits = ['train', 'validation', 'test']
bad_chars = [symbol for symbol in "<>:/\|?!*"]

In [37]:
for split in tqdm(splits):
    with open(os.path.join(superseg_path, f'segmentation_file_{split}.json')) as f:
        data = json.load(f)
    dataset = {'data' : []}
    for idx, current_dialog in enumerate(tqdm(data['dial_data']['tiage'])):
        current_topic_utterances = [] 
        current_topic_id = 0
        for item in current_dialog['turns']:
            if item['topic_id'] == current_topic_id:
                text = ''.join(i for i in item['utterance'].strip() if not i in bad_chars)
                current_topic_utterances.append(text)
            else:
                keywords = kw_model.extract_keywords(' '.join(current_topic_utterances), top_n=top_n,  keyphrase_ngram_range=ngram_range, stop_words='english') 
                keywords = [word[0] for word in keywords]
                dataset['data'].append({"text": "\n".join(current_topic_utterances),
                                        "topic_id": current_topic_id,
                                        "keywords": ', '.join(keywords),
                                        "dialogue_id": idx})
                current_topic_id += 1
                current_topic_utterances = [item['utterance'].strip()]

    with open(f'./data/tiage_{split}.json', 'w') as f:
        json.dump(dataset, f)   

100%|██████████| 3/3 [00:19<00:00,  6.46s/it]


In [8]:
ngram_range = (1, 1)
top_n = 5
qmsum_path = './datasets/QMSum'
splits = ['train', 'validation', 'test']
acadenic_ds = {'train' : [], 'validation': [], 'test': []}

In [10]:
files = os.listdir(os.path.join(qmsum_path, 'Academic', 'train'))
with open(os.path.join(qmsum_path, 'Academic', 'train', files[0])) as f:
    data = json.load(f)

In [12]:
data['topic_list']

[{'topic': 'Technical issues', 'relevant_text_span': [['0', '171']]},
 {'topic': 'Transcription pipeline', 'relevant_text_span': [['172', '372']]},
 {'topic': 'Options for carrying out transcription',
  'relevant_text_span': [['373', '877']]},
 {'topic': 'Transcription conventions and interfaces',
  'relevant_text_span': [['878', '1156']]},
 {'topic': 'Time cost of annotation and website',
  'relevant_text_span': [['1157', '1341']]},
 {'topic': 'Electronics', 'relevant_text_span': [['1342', '1744']]}]

In [28]:
current_topic_utterances = []
text = [utter['content'] for utter in data['meeting_transcripts']]
new_dialog = []
for idx, topics in enumerate(data['topic_list']):
    #topic_idx = 
    topic_start_idx, topic_end_idx = int(topics['relevant_text_span'][0][0]), int(topics['relevant_text_span'][0][1]) + 1
    keywords = kw_model.extract_keywords(' '.join(text[topic_start_idx:topic_end_idx]).strip(), 
                                         top_n=top_n,  
                                         keyphrase_ngram_range=ngram_range, 
                                         stop_words='english') 
    keywords = [word[0] for word in keywords]
    new_dialog.append({'text': text[topic_start_idx:topic_end_idx],
                        'topic_id': idx,
                        'keywords': keywords})
    break

acadenic_ds['train'].append(new_dialog)

In [29]:
acadenic_ds

{'train': [[{'text': ['OK , this is one channel . Can you uh , say your name and talk into your mike one at a time ?',
     'This is Eric on channel three , I believe .',
     "OK . Uh , I don't think it 's on there , Jane .",
     'Tasting one two three , tasting .',
     'OK , this is Jane on channel five .',
     "Uh , I still don't see you Jane .",
     'Oh , darn , what am I doing wrong ?',
     'Can you see me on channel four ? Really ?',
     'Yeah , I s',
     'My lucky day .',
     'Uh , screen no , {disfmarker} it is , oh , maybe it just warmed up ?',
     'No .',
     "Oh , darn , can you can't see channel five yet ?",
     "Uh , well , the mike isn't close enough to your mouth , so .",
     'Oh , this would be k OK , is that better ?',
     'S uh , try speaking loudly ,',
     'I like the high quality labelling .',
     'so ,',
     'Hello ,',
     'OK , good .',
     'David , can we borrow your labelling machine to improve the quality of the labelling a little bit here ?',

[
    {
        "dial_id": 0,
        "utterances": [
            "hey ! do you love cats ?",
            "hey . . . i am a dog person , i have two",
            "ah that is cool , i have two cats and got a collection of 1000 hats for them !",
            "wow ! ! ! that is a lot lol",
            "yeah , i have a weakness for cats and vanilla ice cream , they are the best !",
            "my weakness is eating when i am bored",
            "what is your favorite season ? mine is winter !",
            "i am a summer girl",
            "have you ever watched the olympics ? i won a gold medal in 1992 !",
            "cool what did you win for ?",
            "it was in the 100 meter dash ! i got 7 seconds !",
            "wow ! ! i just run on my treadmill lol",
            "nothing like a good exercise !",
            "yes and i love the rower machine",
            "row row row your boat gently down the stream , when you see a crocodile . . .",
            "lol nursery r h y m e s"
        ],
        "segments": [
            4,
            2,
            2,
            8
        ],
        "set": "train"
    },

In [14]:
with open("../datasets/superseg/segmentation_file_test.json", 'r') as f:
    data = json.load(f)['dial_data']['superseg-v2']

In [15]:
data

[{'dial_id': '0db217e1e912f2c31ce624690f02afa4',
  'turns': [{'da': 'query_condition',
    'role': 'user',
    'turn_id': 1,
    'utterance': 'I need help planning my Social Security benefits for my survivors.',
    'topic_id': 0,
    'segmentation_label': 0},
   {'da': 'query_condition',
    'role': 'agent',
    'turn_id': 2,
    'utterance': 'Are you currently planning for your future?',
    'topic_id': 0,
    'segmentation_label': 0},
   {'da': 'response_positive',
    'role': 'user',
    'turn_id': 3,
    'utterance': 'Yes, I am.',
    'topic_id': 0,
    'segmentation_label': 0},
   {'da': 'respond_solution',
    'role': 'agent',
    'turn_id': 4,
    'utterance': 'you should think about what your family would need if you die now. Social Security can help your family if you have earned enough credits through your work.',
    'topic_id': 0,
    'segmentation_label': 1},
   {'da': 'query_solution',
    'role': 'user',
    'turn_id': 5,
    'utterance': 'Could the benefits received by

In [26]:
annot = []
for item in data:
    utterances = []
    segments = []
    seg_cnt = 1
    for ut in item['turns']:
        utterances.append(ut['utterance'])
        if ut['segmentation_label'] == 1:
            segments.append(seg_cnt)
            seg_cnt = 1
        else:
            seg_cnt += 1
    segments.append(seg_cnt - 1)
    dct = {"dial_id": item['dial_id'],
           "utterances": utterances,
           "segments": segments,
           "set": 'test'}
    annot.append(dct)

In [29]:
with open('/home/tim/repos/dynamic-topic-modeling/datasets/eval/superseg_small.json', 'w') as f:
    json.dump(annot, fp=f)
